# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F
import torch.optim as optim

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,20,5)         # 10, 24x24
        self.conv2=nn.Conv2d(20,128,3)     # 20, 12x12
        #self.conv3=nn.Conv2d(128,256,3,2,1)     # 20, 5*5
        self.fc1=nn.Linear(128*10*10, 1024)
        self.fc2=nn.Linear(1024, 10)
    def forward(self, x):
        in_size=x.size(0)
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  # 12*12
        out = self.conv2(out)  # 10*10
        #out = F.relu(out)
        #out = self.conv3(out)
        out = F.relu(out)
        out = out.view(in_size, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # softmax
        out = F.log_softmax(out, dim=1)

        return out


    
model = SimpleNet().cuda()

# TODO:define loss function and optimiter
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
model.train()
for epoch in range(NUM_EPOCHS):
    batch_idx = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = images.cuda()
        labels = labels.cuda()
        output = model(images)
        optimizer.zero_grad()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0:            # 输出结果
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        batch_idx += 1
        
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
with torch.no_grad():
    correct = 0
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()
        output = model(images)
        pred = output.max(1, keepdim=True)[1]       ### 找到概率最大的下标
        correct += pred.eq(labels.view_as(pred)).sum().item()
    test_accuracy = 100. * correct / len(test_loader.dataset)
    correct = 0
    for images, labels in train_loader:
        images, labels = images.cuda(), labels.cuda()
        output = model(images)
        pred = output.max(1, keepdim=True)[1]       ### 找到概率最大的下标
        correct += pred.eq(labels.view_as(pred)).sum().item()
    train_accuracy = 100. * correct / len(train_loader.dataset)

  6%|████▊                                                                            | 28/468 [00:05<00:30, 14.46it/s]

Train Epoch: 0 [3712/60000 (6%)]	Loss: 0.223668


 12%|██████████                                                                       | 58/468 [00:07<00:22, 18.44it/s]

Train Epoch: 0 [7552/60000 (13%)]	Loss: 0.176074


 19%|███████████████▍                                                                 | 89/468 [00:09<00:21, 17.78it/s]

...

 90%|███████████████████████████████████████████████████████████████████████▌        | 419/468 [00:27<00:03, 14.72it/s]

Train Epoch: 9 [53632/60000 (90%)]	Loss: 0.023734


 96%|████████████████████████████████████████████████████████████████████████████▊   | 449/468 [00:28<00:01, 16.14it/s]

Train Epoch: 9 [57472/60000 (96%)]	Loss: 0.001772


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:30<00:00, 15.55it/s]


#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training accuracy is: {}%'.format(train_accuracy))
print('Testing accuracy is: {}%'.format(test_accuracy))

Training accuracy is: 99.56%
Testing accuracy is: 98.66%
